Breakpoint function on hooked CFG dispatch function to comment the source and add xref to destination

In [ ]:
import ida_dbg
import ida_xref
import ida_bytes
import ida_ua
import ida_allins
import ida_funcs
import ida_segment

call_src_nxt = ida_bytes.get_qword(ida_dbg.get_reg_val("rsp"))
call_src = ida_bytes.prev_head(call_src_nxt, 0)
dest_addr = ida_dbg.get_reg_val("rax")
dest_name = ida_funcs.get_func_name(dest_addr)

text_segment = ida_segment.get_segm_by_name(".text")
if not (text_segment.start_ea < call_src < text_segment.end_ea):
    return False

instruction = ida_ua.insn_t()
ida_ua.decode_insn(instruction, call_src)

if instruction.itype == ida_allins.NN_callni or instruction.itype == ida_allins.NN_call:
    ida_xref.add_cref(call_src, dest_addr, ida_xref.XREF_USER | ida_xref.fl_CN)
    ida_bytes.set_cmt(call_src, dest_name, False)
    print(f"CALL Xref from {hex(call_src)} to {hex(dest_addr)} ({dest_name})")
else:
    print(f"Invalid instruction type for branch from {hex(call_src)} to {hex(dest_addr)} ({dest_name}) return to {hex(call_src_nxt)}")
    return True

return False

Loop through all code and remove color

In [ ]:
import ida_ida
import ida_nalt
import ida_bytes
import ida_idaapi

start_ea = ida_ida.inf_get_min_ea()
end_ea = ida_ida.inf_get_max_ea()

if ida_bytes.is_code(ida_bytes.get_full_flags(start_ea)):
    ea = start_ea
else:
    ea = ida_bytes.next_head(start_ea, end_ea)

while ea != ida_idaapi.BADADDR and ea < end_ea:
    ida_nalt.set_item_color(ea, 0xFFFFFF)
    ea = ida_bytes.next_head(ea, end_ea)

Loop through all code and colorize all call and indirect jmp instructions

In [ ]:
import ida_ida
import ida_ua
import ida_nalt
import ida_bytes
import ida_idaapi

start_ea = ida_ida.inf_get_min_ea()
end_ea = ida_ida.inf_get_max_ea()
ea = ida_bytes.next_head(start_ea, end_ea)

while ea != ida_idaapi.BADADDR and ea < end_ea:
    instruction = ida_ua.insn_t()
    if ida_ua.decode_insn(instruction, ea):
        mnemonic = ida_ua.print_insn_mnem(ea)
        if mnemonic and mnemonic.lower() == "call":
            ida_nalt.set_item_color(ea, 0xFFFFAA)
        elif mnemonic and mnemonic.lower() == "jmp":
            if instruction.ops[0].type in [ida_ua.o_phrase, ida_ua.o_displ, ida_ua.o_reg]:
                ida_nalt.set_item_color(ea, 0xFFFFAA)
    ea = ida_bytes.next_head(ea, end_ea)

Reload plugin

In [ ]:
import testplugin
import importlib
importlib.reload(testplugin)
testplugin.PLUGIN_ENTRY().run(0)

Diablo Code

In [ ]:
import ida_idd

read_book = ida_idd.Appcall.proto(0x7FF76BBF0B40, "__int64 __fastcall sub_7FF76BBF0B40(int64_t, int64_t, int64_t, int64_t);")
for spell_index in range(2, 64):
    read_book(0x00, 0x18, spell_index, 0x07)